In [1]:
from importLib import import_from_github
import_from_github(r"https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/extras/exercises/03_pytorch_computer_vision_exercises.ipynb")



c:\Users\22gusgam\Desktop\k\PyTorch\03\exercises/03_pytorch_computer_vision_exercises.ipynb doesn't exist, download
<class 'str'>
